In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from d2l import torch as d2l
#theano is the oldest ML library
#A blovk is a network segment.  Can be layer, series of layers, entire network etc

In [15]:
#example of a block
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[-0.0021, -0.4248, -0.0930,  0.1545, -0.0131, -0.1377, -0.1495,  0.1105,
         -0.0314,  0.0635],
        [-0.0618, -0.2305, -0.0383,  0.1242,  0.0056, -0.1792, -0.0558,  0.0121,
          0.0390,  0.1127]], grad_fn=<AddmmBackward>)

In [17]:
#sequential is a daisy chain of blocks in a specific order
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0431,  0.2674,  0.1321,  0.0517,  0.1473, -0.2153,  0.0237, -0.1868,
         -0.1735, -0.2019],
        [ 0.0499,  0.1563,  0.0991,  0.0448,  0.0785, -0.1681,  0.0169, -0.0891,
         -0.2675, -0.2638]], grad_fn=<AddmmBackward>)

In [18]:
#This is how you will pretty much always make a network
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(-0.2160, grad_fn=<SumBackward0>)

In [19]:
#How to access parameters
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0685],
        [-0.2438]], grad_fn=<AddmmBackward>)

In [33]:
print(net)
print(net[2])
#get an overview
print(net[2].state_dict())
#not a parameter? is param.data?
print(net[2].state_dict()["bias"])
#parameter
print(net[2].bias)
#same as state dict
print(net[2].bias.data)
print(net[2].bias.requires_grad)
#can get gradient value too (not trained yet though)
print(net[2].bias.grad)


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)
Linear(in_features=8, out_features=1, bias=True)
OrderedDict([('weight', tensor([[-0.0419,  0.1816,  0.2493,  0.1570,  0.1082,  0.0817,  0.0870, -0.2020]])), ('bias', tensor([-0.3203]))])
tensor([-0.3203])
Parameter containing:
tensor([-0.3203], requires_grad=True)
tensor([-0.3203])
True
None


In [43]:
#we can get all parameters with
print (net.named_parameters)
for ip,np in enumerate(net.named_parameters()):
    print (ip,np[0],np[1])
net.state_dict()['2.bias']


<bound method Module.named_parameters of Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)>
0 0.weight Parameter containing:
tensor([[ 0.2759,  0.4305, -0.3554, -0.3009],
        [ 0.2732,  0.1758,  0.0812,  0.2077],
        [-0.0738,  0.2629,  0.0472,  0.4754],
        [-0.1195,  0.2767, -0.1444,  0.1801],
        [ 0.3931,  0.3913,  0.4467,  0.1970],
        [ 0.2864, -0.1781, -0.3565,  0.3385],
        [-0.2291, -0.1260, -0.1201, -0.0936],
        [-0.2741,  0.0741, -0.4921, -0.1266]], requires_grad=True)
1 0.bias Parameter containing:
tensor([ 0.3111, -0.0382, -0.4193, -0.1520, -0.0104, -0.0703, -0.4638, -0.1414],
       requires_grad=True)
2 2.weight Parameter containing:
tensor([[-0.0419,  0.1816,  0.2493,  0.1570,  0.1082,  0.0817,  0.0870, -0.2020]],
       requires_grad=True)
3 2.bias Parameter containing:
tensor([-0.3203], requires_grad=True)


tensor([-0.3203])

In [46]:
#but what about blocks?
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # can name blocks like this!
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [48]:
#Can share parameters through layers
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
